<a href="https://colab.research.google.com/github/davidlealo/100profes/blob/master/prueba_usuarios_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Simulación de un chat con memoria de usuario

# Memoria para almacenar el contexto por usuario (simulación en memoria)
user_sessions = {}

# Función para manejar la interacción con el usuario
def chat_with_user(user_id, message):
    # Recuperar el contexto del usuario si existe
    if user_id not in user_sessions:
        user_sessions[user_id] = {"context": []}
        print(f"[INFO] Nueva sesión creada para el usuario: {user_id}")

    context = user_sessions[user_id]["context"]

    # Crear la respuesta basándonos en el contexto
    response = f"Hola, {user_id}. Me dijiste: {message}. "
    if context:
        response += f"Recuerdo que también mencionaste: {' | '.join(context)}."
    else:
        response += "Es nuestra primera conversación."

    # Actualizar el contexto con el mensaje actual
    context.append(message)
    user_sessions[user_id]["context"] = context

    return response

# Probar el sistema de chat
print("Prueba de Chat con Memoria")
while True:
    user_id = input("Ingresa tu ID de usuario (escribe 'salir' para terminar): ")
    if user_id.lower() == "salir":
        break
    message = input("Tú: ")
    response = chat_with_user(user_id, message)
    print(f"Chatbot: {response}")


Prueba de Chat con Memoria
Ingresa tu ID de usuario (escribe 'salir' para terminar): 1
Tú: David
[INFO] Nueva sesión creada para el usuario: 1
Chatbot: Hola, 1. Me dijiste: David. Es nuestra primera conversación.
Ingresa tu ID de usuario (escribe 'salir' para terminar): 1
Tú: David
Chatbot: Hola, 1. Me dijiste: David. Recuerdo que también mencionaste: David.
Ingresa tu ID de usuario (escribe 'salir' para terminar): 1
Tú: Hola
Chatbot: Hola, 1. Me dijiste: Hola. Recuerdo que también mencionaste: David | David.
Ingresa tu ID de usuario (escribe 'salir' para terminar): 2
Tú: Bea
[INFO] Nueva sesión creada para el usuario: 2
Chatbot: Hola, 2. Me dijiste: Bea. Es nuestra primera conversación.
Ingresa tu ID de usuario (escribe 'salir' para terminar): salir
